<a href="https://colab.research.google.com/github/Sai3851/Data-driven-Defect-Prediction/blob/main/Copy_of_AEEM_JIRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Loading each dataset
eq_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/EQ.csv')
jdt_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/JDT.csv')
lc_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/LC.csv')
ml_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/ML.csv')
activemq_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/JIRA/activemq-5.0.0.csv')
derby_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/JIRA/derby-10.5.1.1.csv')
hbase_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/JIRA/hbase-0.94.0.csv')
hive_data = pd.read_csv('/content/drive/MyDrive/Defect_Prediction_Datasets/JIRA/hive-0.9.0.csv')
# Displaying the first few rows of each dataset to check structure
print("EQ Data:\n", eq_data.head())
print("activemq Data:\n", activemq_data.head())


EQ Data:
    ck_oo_numberOfPrivateMethods  LDHH_lcom  LDHH_fanIn  \
0                             3   0.002547    0.002555   
1                            37   0.008643    0.004756   
2                             3   0.001479    0.009143   
3                            10   0.005642    0.005395   
4                             1   0.001350    0.000000   

   numberOfNonTrivialBugsFoundUntil:  WCHU_numberOfPublicAttributes  \
0                                  4                           0.00   
1                                 71                           0.00   
2                                  5                           1.01   
3                                 38                           0.00   
4                                  1                           1.01   

   WCHU_numberOfAttributes  CvsWEntropy  LDHH_numberOfPublicMethods  \
0                     3.04     0.393707                    0.003049   
1                    14.37     2.093750                    0.001481   
2

In [ ]:
# Applying fillna() only to numeric columns
eq_data.fillna(eq_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
jdt_data.fillna(jdt_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
lc_data.fillna(lc_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
ml_data.fillna(ml_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)
activemq_data.fillna(activemq_data.select_dtypes(include=['float64', 'int64']).mean(), inplace=True)

In [ ]:
#feature extraction
# Defining the number of components for PCA
n_components = 10  # Adjust as needed

# PCA for source data
pca_source = PCA(n_components=n_components)
source_data_reduced = pca_source.fit_transform(source_data.select_dtypes(include=['float64', 'int64']))

# PCA for target data
pca_target = PCA(n_components=n_components)
target_data_reduced = pca_target.fit_transform(target_data.select_dtypes(include=['float64', 'int64']))



In [ ]:
#  'class' is our source label and 'RealBug' is target label
y_source = source_data['class'].reset_index(drop=True)  # Source target labels
y_target = target_data['RealBug'].reset_index(drop=True)  # Target target labels

# `source_data_reduced` and `target_data_reduced` are already the feature sets from PCA
X_source = pd.DataFrame(source_data_reduced)  # Source features
X_target = pd.DataFrame(target_data_reduced)  # Target features



In [ ]:
from imblearn.over_sampling import SMOTE

# Applying SMOTE if there's class imbalance
smote = SMOTE()
X_source_balanced, y_source_balanced = smote.fit_resample(X_source, y_source)


In [ ]:
#Ensuring the training data is in the format TensorFlow expects (numpy arrays).
import numpy as np
X_source_balanced = np.array(X_source_balanced)
y_source_balanced = np.array(y_source_balanced)


In [ ]:
import numpy as np

# Replace 'buggy' with 1 and 'clean' with 0 in y_source_balanced
y_source_balanced = np.where(y_source_balanced == 'buggy', 1, 0).astype(float)

# Replace 'TRUE' with 1 and 'FALSE' with 0 in y_target
y_target = np.where(y_target == True, 1, 0).astype(float)



In [ ]:
from tensorflow.keras import layers, models

# Defining and compiling the encoder network
encoder = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_source_balanced.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),  # Latent space
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

encoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
history = encoder.fit(X_source_balanced, y_source_balanced, epochs=20, validation_split=0.2)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.4338 - loss: 0.7215 - val_accuracy: 0.4872 - val_loss: 0.6727
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7339 - loss: 0.6228 - val_accuracy: 0.5385 - val_loss: 0.6325
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7198 - loss: 0.5835 - val_accuracy: 0.5385 - val_loss: 0.6312
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7584 - loss: 0.5347 - val_accuracy: 0.5641 - val_loss: 0.6468
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7275 - loss: 0.5178 - val_accuracy: 0.5769 - val_loss: 0.6693
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7299 - loss: 0.5141 - val_accuracy: 0.6154 - val_loss: 0.6737
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7404 - loss: 0.4893 - val_accuracy: 0.6282 - val_loss: 0.6678
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7452 - loss: 0.4628 - val_accuracy: 0.6410 - val_loss: 0.6568
Ep

In [ ]:
y_target = y_target.reshape(-1, 1)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Predicting and evaluating on the target dataset
y_pred = (encoder.predict(X_target) > 0.5).astype(int)
test_loss, test_accuracy = encoder.evaluate(X_target, y_target)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Print detailed classification metrics
print(classification_report(y_target, y_pred))

# Calculate and display AUC
auc_score = roc_auc_score(y_target, y_pred)
print("AUC:", auc_score)


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8408 - loss: 25.7767
Test Accuracy: 0.8615
              precision    recall  f1-score   support

         0.0       0.88      0.96      0.92      1591
         1.0       0.61      0.31      0.41       293

    accuracy                           0.86      1884
   macro avg       0.74      0.64      0.67      1884
weighted avg       0.84      0.86      0.84      1884

AUC: 0.6381405216630235


In [ ]:

# Predicting and evaluating for Baseline Model
y_pred_baseline = (encoder.predict(X_target) > 0.5).astype(int)
pd_base, pf_base, f1_base, g_mean_base, auc_base = calculate_metrics(y_target, y_pred_baseline)
print(f"Baseline Model - PD: {pd_base}, PF: {pf_base}, F1: {f1_base}, G-mean: {g_mean_base}, AUC: {auc_base}")


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Baseline Model - PD: 0.31399317406143346, PF: 0.03771213073538655, F1: 0.4134831460674157, G-mean: 0.5496833838049042, AUC: 0.6381405216630235


In [ ]:
import tensorflow as tf

def mmd_loss(source, target, gamma=1.0):
    # Computing pairwise distances between source and target samples
    xx = tf.matmul(source, tf.transpose(source))
    yy = tf.matmul(target, tf.transpose(target))
    xy = tf.matmul(source, tf.transpose(target))

    # Computing kernels
    rx = tf.reduce_sum(tf.square(source), axis=1, keepdims=True)
    ry = tf.reduce_sum(tf.square(target), axis=1, keepdims=True)
    K = tf.exp(-gamma * (rx - 2 * xx + tf.transpose(rx)))
    L = tf.exp(-gamma * (ry - 2 * yy + tf.transpose(ry)))
    P = tf.exp(-gamma * (rx - 2 * xy + tf.transpose(ry)))

    # Calculating MMD
    return tf.reduce_mean(K) + tf.reduce_mean(L) - 2 * tf.reduce_mean(P)


In [ ]:
y_source_balanced = y_source_balanced.reshape(-1, 1)  # Convert to 2D shape


In [ ]:

# Training loop with MMD
def train_with_mmd(encoder, source_data, y_source, target_data, epochs=20, gamma=1.0):
    optimizer = tf.keras.optimizers.Adam()

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            # Forward pass
            source_features = encoder(source_data, training=True)
            target_features = encoder(target_data, training=True)
            predictions = encoder(source_data, training=True)

            # Calculating classification and MMD loss
            classification_loss = tf.keras.losses.binary_crossentropy(y_source, predictions)
            mmd = mmd_loss(source_features, target_features, gamma)
            total_loss = classification_loss + mmd

        # Backpropagation
        gradients = tape.gradient(total_loss, encoder.trainable_variables)
        optimizer.apply_gradients(zip(gradients, encoder.trainable_variables))

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss.numpy()}, MMD Loss: {mmd.numpy()}")

# Run training with MMD
train_with_mmd(encoder, X_source_balanced, y_source_balanced, X_target)


Epoch 1/20, Loss: [0.86536163 0.18757826 1.4112343  0.20491767 1.0519109  0.2400837
 0.2972868  0.25156307 0.3389148  0.61569655 0.7681568  0.28683656
 0.26164716 0.24579933 0.30357295 0.25156307 0.5302433  0.25056854
 0.32440525 0.2400837  0.71354294 0.6762472  0.5792942  0.2400837
 0.40926626 0.6943637  0.2720039  0.2453592  0.89465475 0.20149757
 0.25056854 0.2453592  1.5570661  0.25056854 0.28853127 0.6344657
 0.5528054  0.65704226 0.25056854 0.729042   0.7908532  0.29288104
 0.42879337 0.7512222  1.7210934  0.32751995 0.27483487 0.6441229
 0.8368297  0.3551428  0.8181593  0.7857406  0.8875458  0.3553021
 1.9092674  0.5717553  0.7933912  0.63028324 0.2453592  0.6877001
 0.2720039  1.7655346  0.9712253  0.34052974 0.18741986 0.51866335
 0.27093104 0.32434267 0.5652845  2.095127   0.55169535 0.30413902
 0.2752836  0.40263036 0.61027133 1.5648558  0.5473056  0.2968179
 2.5273805  0.34096074 0.39573622 0.58138514 1.0582007  0.8018345
 0.27790186 0.48063034 1.0050082  0.68562984 0.42675

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Predicting on the target dataset
y_pred = (encoder.predict(X_target) > 0.5).astype(int)
test_loss, test_accuracy = encoder.evaluate(X_target, y_target)
print(f"Test Accuracy with MMD: {test_accuracy:.4f}")

# Print detailed classification metrics
print("Classification Report:")
print(classification_report(y_target, y_pred))

# Calculating and display AUC
auc_score = roc_auc_score(y_target, y_pred)
print("AUC with MMD:", auc_score)


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8246 - loss: 29.3039
Test Accuracy with MMD: 0.8323
Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.88      0.90      1591
         1.0       0.47      0.56      0.51       293

    accuracy                           0.83      1884
   macro avg       0.69      0.72      0.70      1884
weighted avg       0.85      0.83      0.84      1884

AUC with MMD: 0.7224876277182016


In [ ]:
# Predicting and evaluating for MMD Model
y_pred_mmd = (encoder.predict(X_target) > 0.5).astype(int)
pd_mmd, pf_mmd, f1_mmd, g_mean_mmd, auc_mmd = calculate_metrics(y_target, y_pred_mmd)
print(f"MMD Model - PD: {pd_mmd}, PF: {pf_mmd}, F1: {f1_mmd}, G-mean: {g_mean_mmd}, AUC: {auc_mmd}")


59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MMD Model - PD: 0.5631399317406144, PF: 0.1181646763042112, F1: 0.5108359133126935, G-mean: 0.7046961643094909, AUC: 0.7224876277182016
